In [ ]:
!pip install ../input/nlpaug/nlpaug-1.1.10-py3-none-any.whl
#!pip install ../input/pandarallel151whl/pandarallel-1.5.1-py3-none-any.whl

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, DebertaTokenizer, BertTokenizer, TFAutoModel
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import TFDebertaModel
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

In [ ]:
# 02/08/2022
#https://www.kaggle.com/code/bharadwajvedula/feedback-prize-tpu-roberta-inference
#https://www.kaggle.com/code/bharadwajvedula/feedback-prize-tpu-roberta-training-fold-4

#https://www.kaggle.com/code/imvision12/tensorflow-feedback-bert-baseline

In [ ]:
#https://www.kaggle.com/competitions/feedback-prize-effectiveness/discussion/339739

In [ ]:
TEST_DIR = "../input/feedback-prize-effectiveness"
TRAIN_FOLDS_DIR = '../input/feedbacktrainfolds'

# Stratified KFold

In [ ]:
# INPUT_DIR = "../input/feedback-prize-effectiveness"

In [ ]:
# train = pd.read_csv(f"{INPUT_DIR}/train.csv")
# test = pd.read_csv(f"{INPUT_DIR}/test.csv")
# submission = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv")

In [ ]:
# train['discourse_effectiveness'].value_counts()

In [ ]:
# import nlpaug.augmenter.word as naw
# aug = naw.ContextualWordEmbsAug('../input/huggingface-bert-variants/bert-base-cased/bert-base-cased', device='cuda')

In [ ]:
# def augment_text(text):
#     augmented_data = aug.augment(text)
#     return augmented_data
# to_augment = train[train['discourse_effectiveness'].isin(['Effective','Ineffective'])]
# to_augment['augmented_text'] = to_augment['discourse_text'].apply(augment_text)

In [ ]:
# to_augment.columns

In [ ]:
# to_augment = to_augment[['discourse_id','essay_id','augmented_text','discourse_type','discourse_effectiveness']]
# to_augment.rename({"augmented_text":'discourse_text'}, axis=1, inplace=True)

In [ ]:
# train = pd.concat([train, to_augment], axis=0).reset_index(drop=True)

In [ ]:
# train['discourse_effectiveness'].value_counts()

In [ ]:
# train['kfold'] = -1
# kf = StratifiedKFold(n_splits=5)
# for f, (t_, v_) in enumerate(kf.split(X=train, y=train['discourse_effectiveness'])):
#     train.loc[v_, 'kfold'] = f
# train.to_csv("train_folds_aug.csv",index=False)

# Model

In [ ]:
train = pd.read_csv(f"{TRAIN_FOLDS_DIR}/train_folds_aug.csv")
test = pd.read_csv(f"{TEST_DIR}/test.csv")
submission = pd.read_csv(f"{TEST_DIR}/sample_submission.csv")

In [ ]:
train['discourse_effectiveness'].value_counts()

In [ ]:
train['discourse_text'][0]

In [ ]:
# Normalize do not give better result
# def normalise(text):
#     text = text.lower()
#     text = text.strip()
#     text = re.sub("\n", " ", text)
#     return text

# train['discourse_text'] = train['discourse_text'].apply(normalise)
# train['discourse_type'] = train['discourse_type'].apply(normalise)

# ## TEST
# test['discourse_text'] = test['discourse_text'].apply(normalise)
# test['discourse_type'] = test['discourse_type'].apply(normalise)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("../input/huggingface-bert-variants/bert-base-cased/bert-base-cased")

## labels

In [ ]:
label_mapping = {"Adequate":0, "Effective":1, "Ineffective":2}
train['discourse_effectiveness'] = train['discourse_effectiveness'].map(label_mapping)

In [ ]:
train['discourse_effectiveness'].value_counts()

## Tokenizer

In [ ]:
train['inputs'] = train.discourse_type + tokenizer.sep_token +train.discourse_text
test['inputs'] = test.discourse_type + tokenizer.sep_token +test.discourse_text

In [ ]:
tokenizer.encode_plus(train['inputs'][0], add_special_tokens=True,
                              padding='max_length',max_length=256).keys()

In [ ]:
train.shape[0]

In [ ]:
def tokenizer_encode(texts, tokenizer):
    
    input_ids = np.zeros((train.shape[0], 256))
    token_type_ids = np.zeros((train.shape[0], 256))
    attention_mask_ids = np.zeros((train.shape[0], 256))
    
    for e,text in enumerate(texts):
        tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True,
                              padding='max_length',max_length=256,truncation= True)
        input_ids[e,:] = tokenized_text['input_ids']
        token_type_ids[e,:] = tokenized_text['token_type_ids']
        attention_mask_ids[e,:] = tokenized_text['attention_mask']

    return input_ids, token_type_ids, attention_mask_ids

In [ ]:
X_train = tokenizer_encode(train['inputs'].astype(str), tokenizer)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
def build_model(fold, bert_model, max_len=256):  
    callbacks = tf.keras.callbacks.ModelCheckpoint(f'./model{fold}_weights.h5',
                                                   save_weights_only=True, save_best_only=True)
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = tf.keras.layers.Dropout(.1)(clf_output)
    out = tf.keras.layers.Dense(3, activation='softmax')(clf_output)
    
    model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model, callbacks

In [ ]:
bert_model = (transformers.TFBertModel.from_pretrained('../input/huggingface-bert-variants/bert-base-cased/bert-base-cased'))

# Model Training

In [ ]:
train['discourse_effectiveness'].value_counts()

In [ ]:
# model_list = []
# for fold in range(5):
#     train_fold = train[train['kfold'] != fold]
#     valid_fold = train[train['kfold'] == fold]
    
#     X_train_dataset = (X_train[0][train_fold.index.to_list(),:], X_train[1][train_fold.index.to_list(),:],
#               X_train[2][train_fold.index.to_list(),:])
#     X_valid_dataset = (X_train[0][valid_fold.index.to_list(),:], X_train[1][valid_fold.index.to_list(),:],
#               X_train[2][valid_fold.index.to_list(),:])
#     train_dataset = (
#         tf.data.Dataset
#         .from_tensor_slices((X_train_dataset, train_fold['discourse_effectiveness'].values))
#         .repeat()
#         .shuffle(2048)
#         .batch(16)
#         .prefetch(AUTO)
#     )
#     valid_dataset = (
#         tf.data.Dataset
#         .from_tensor_slices((X_valid_dataset, valid_fold['discourse_effectiveness']))
#         .batch(16)
#         .cache()
#         .prefetch(AUTO)
#     )
#     model, modelcheckpt = build_model(fold, bert_model, 256)
#     train_history = model.fit(train_dataset,steps_per_epoch=200,validation_data=valid_dataset,epochs=10,
#                              callbacks=[modelcheckpt])
#     model_list.append(model)

In [ ]:
# fold0 0.8151
# fold1 0.7656
# fold2 0.5965
# fold3 0.5155
# fold4 0.65

In [ ]:
label_mapping = {"Adequate":0, "Effective":1, "Ineffective":2}

In [ ]:
def test_tokenizer_encode(texts, tokenizer):
    
    input_ids = np.zeros((test.shape[0], 256))
    token_type_ids = np.zeros((test.shape[0], 256))
    attention_mask_ids = np.zeros((test.shape[0], 256))
    
    for e,text in enumerate(texts):
        tokenized_text = tokenizer.encode_plus(text, add_special_tokens=True,
                              padding='max_length',max_length=256,truncation= True)
        input_ids[e,:] = tokenized_text['input_ids']
        token_type_ids[e,:] = tokenized_text['token_type_ids']
        attention_mask_ids[e,:] = tokenized_text['attention_mask']

    return input_ids, token_type_ids, attention_mask_ids

In [ ]:
X_test = test_tokenizer_encode(test['inputs'].astype(str), tokenizer)

In [ ]:
prediction_list = []
for fold in range(5):
    X_test_dataset = (X_test[0][:,:], X_test[1][:,:],
              X_test[2][:,:])
    test_dataset = (
        tf.data.Dataset
        .from_tensor_slices((X_test_dataset))
        .batch(16)
        .cache()
        .prefetch(AUTO)
    )
    model, callback_option = build_model(fold, bert_model, 256)
    model.load_weights(f"../input/bertcasedfeedback/model{fold}_weights.h5")
    prediction = model.predict([X_test[0],X_test[1],X_test[2]])
    prediction_list.append(prediction)

In [ ]:
predictions = prediction_list[0]+prediction_list[1]+ \
              prediction_list[2]+prediction_list[3]+ \
              prediction_list[4]
predictions = (predictions / 5)
predictions

In [ ]:
test['Adequate'] = predictions[:,0]
test['Effective'] = predictions[:,1]
test['Ineffective'] = predictions[:,2]

In [ ]:
test[submission.columns]

In [ ]:
test[submission.columns].to_csv("submission.csv", index=False)